In [ ]:
class HoltWinters:
    """
    Holt-Winters model
    # series - initial time series
    # slen - length of a season
    # alpha, beta, gamma - Holt-Winters model coefficients
    # n_preds - predictions 
    """   
    def __init__(self, series, slen, alpha, beta, gamma, n_preds):
        self.series = series
        self.slen = slen
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        
    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i+self.slen] - self.series[i]) / self.slen
        return sum / self.slen  
    
    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series)/self.slen)
        # let's calculate season averages
        for j in range(n_seasons):
            # seasonal mean of streamflow
            season_averages.append(sum(self.series[self.slen*j:self.slen*(j+1)])/float(self.slen))
        # let's calculate initial values
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += self.series[self.slen*j+i]-season_averages[j]
            seasonals[i] = sum_of_vals_over_avg/n_seasons
        return seasonals   
    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        seasonals = self.initial_seasonal_components()        
        for i in range(len(self.series)+self.n_preds):
            if i == 0: # components initialization
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i%self.slen])                
                continue                
            if i >= len(self.series): # predicting
                m = i - len(self.series) + 1
                val = self.result[i-1]
                last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                self.result.append((smooth + m*trend) + seasonals[i%self.slen])                
                # when predicting we increase uncertainty on each step
            else:
                val = self.series[i-1]
                last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                self.result.append(smooth+trend+seasonals[i%self.slen])                
            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i%self.slen])

In [ ]:
from sklearn.metrics import mean_squared_error
def Train_Score(params, series, slen, validsize, loss_function=mean_squared_error):
    """
        Returns error         
        param   - parameter for optimization
        series   - timeseries dataset
        validsize- size of validation dataset
    """
    values = series.values
    alpha, beta, gamma = params
    
    training_dataset =
    validation_dataset = 
    model_result = 
    error = loss_function(...,...)
    return error

In [ ]:
%%time
#print CPU times and Wall time
from scipy.optimize import minimize     #for optimization

# prediction horizon is the same as the size of test dataset and validation dataset
predicts = 5
# split dataset into (training+validation) and test dataset
datatrain = ...
# initializing model parameters alpha, beta and gamma
x_iguess = [0.7, 0.1, 0.3] 
# season length is assumed to be 13 days
slength  = 13

# Minimizing the value of loss function
opt = minimize(...)

# take optimal values
alpha_final, beta_final, gamma_final = opt.x
print('alpha', 'beta ', 'gamma')
print("{:.3f}".format(alpha_final), "{:.3f}".format(beta_final),"{:.3f}".format(gamma_final))

# forecasting for the next (predicts) days with optimal parameters
model = HoltWinters(...)
model.triple_exponential_smoothing()

In [ ]:
# Report the error
MSE_train = Train_Score(...)
print('MSE of training Process:')
print("{:.2f}".format(MSE_train))
MSE_test = mean_squared_error(...)
print('MSE of testing Process:')
print("{:.2f}".format(MSE_test))

In [ ]:
# Plot the result
plt.plot(Time, Daily.discharge)
plt.plot(Time,model.result)
ax.set(xlabel='Date', 
       ylabel='Discharge Value (cfs)',
       title='Wabash River at Lafayette Station 2019');
plt.show()